In [11]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

folder = '/content/drive/MyDrive/Colab Notebooks/P2-Lasso SAR/'

sdf = pd.read_csv(folder+'data/SP500_Price.csv', index_col=0)
ndt = np.array(sdf)
df = 100*(np.log(ndt[1:,:])-np.log(ndt[:-1,:])) # log return * 100: to prevent the scale too small
print(df.shape)
print(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(5287, 331)
[[ -1.60744039  -0.52195881  -2.67230563 ...  -3.48975443   1.09475412
   -0.2879567 ]
 [ -1.33850104  -1.74501298 -13.35313926 ...  -0.09350326  -0.9744577
   -1.72848819]
 [  4.72871088  -0.10363177  -2.40975516 ...   1.20877      0.49692665
   -1.3556249 ]
 ...
 [ -0.39882623   0.83256891  -1.95128142 ...  -2.42763331   0.16356326
    2.0918475 ]
 [  1.95974491   0.61204784  -0.9901071  ...   1.62495494  -0.07006033
    3.57473696]
 [  0.98413011   0.23247474   0.         ...   1.35687292   0.71389919
   -2.4129048 ]]


In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso


def nodewise_regression(X):
    T, N = X.shape
    precision_matrix = np.zeros((N, N))

    temp_list = []

    for i in range(N):
        y_i = X[:, i]
        X_i = np.delete(X, i, axis=1)

        lasso = Lasso(alpha=3)

        lasso.fit(X_i, y_i)

        coefficients = lasso.coef_
        intercept = lasso.intercept_

        coefficients = np.insert(coefficients, i, 0)

        temp_list.append(coefficients)

    W = np.array(temp_list)

    final_W = (W + W.T) / 2

    return final_W

# Example usage
np.random.seed(0)

X = df

final_W = nodewise_regression(X)

pd.DataFrame(final_W).to_csv(folder+'/lasso_sar_w.csv', index=False)


In [13]:
temp = pd.read_csv(folder+'lasso_sar_w.csv')
temp

,0,1,2,3,4,5,6,7,8,9,...,321,322,323,324,325,326,327,328,329,330
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
nonzero_counts = temp.astype(bool).sum(axis=0)
print('Total nonzero: ', nonzero_counts.sum())
print(nonzero_counts)


Total nonzero:  1002
0      1
1      0
2      0
3      2
4      5
      ..
326    2
327    0
328    2
329    0
330    8
Length: 331, dtype: int64
